<a href="https://colab.research.google.com/github/grupo274/pre-projeto-i2a2/blob/main/I2A2_firstAgent_devstral_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load API key from file

In [1]:
#file_path = 'openrouter_api_key.txt'
api_key_path = 'devstral_small_api_key.txt'

try:
  with open(api_key_path, 'r') as file:
    API_KEY = file.read().strip()
    print(f"The file was successfully loaded.")
except FileNotFoundError:
  print(f"Error: The file '{api_key_path}' was not found.")
except Exception as e:
  print(f"An error occurred: {e}")

The file was successfully loaded.


# Functions that implement the tools

In [2]:
import zipfile
import pandas as pd
import os, json

# Define the functions that implement the tools
def unzip_file(zip_path: str):
  """
  Unzips a given file.

  Args:
    zip_path: The name of the ZIP file to unzip.

  Returns:
    A list of the names of the extracted files, or an error message.
  """
  if zip_path.endswith('.zip'):
    try:
      print(f"Attempting to unzip: {zip_path}")
      with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        extracted_files = zip_ref.namelist()
        zip_ref.extractall()
        print(f"Successfully unzipped {zip_path}. Extracted files: {extracted_files}")
        return json.dumps({"extracted_files": extracted_files})
    except zipfile.BadZipFile:
      return json.dumps({"error": f"'{zip_path}' is not a valid zip file."})
    except FileNotFoundError:
      return json.dumps({"error": f"The file '{zip_path}' was not found."})
    except Exception as e:
      return json.dumps({"error": f"An error occurred while unzipping '{zip_path}': {str(e)}"})
  else:
    return json.dumps({"error": f"'{zip_path}' is not a zip file."})


def read_csv(file_path: str):
    """Reads a CSV file and returns its content."""
    for filename in os.listdir(file_path):
      if filename.endswith('.csv'):
        csv_path = os.path.join(file_path, filename)
        try:
          print(f"Attempting to read CSV: {file_path}")
          with open(csv_path, 'r') as f:
            # Read only a sample of the file to avoid overwhelming context
            content = f.read(1000) + "\n... (truncated)" # Read first 1000 characters
            print(f"Successfully read CSV: {csv_path}")
            return json.dumps({"file_content_sample": content})
        except FileNotFoundError:
          return json.dumps({"error": f"File '{file_path}' not found."})
        except Exception as e:
          return json.dumps({"error": f"An error occurred while reading '{file_path}': {str(e)}"})
      else:
        return json.dumps({"error": f"'{filename}' is not a CSV file."})


def analyze_data(file_path: str, analysis_type: str, column: str, top_n: int = None):
  """
  Analyzes a dataframe loaded from a CSV file based on the provided analysis type and column.

  Args:
    file_path: The path to the CSV file.
    analysis_type: The type of analysis to perform (e.g., 'most frequent item', 'total volume', 'average', 'top items').
    column: The column to perform the analysis on.
    top_n: The number of top items to return for 'top items' analysis (required for 'top items').

  Returns:
    dict: The result of the analysis or an error message.
  """
  try:
    print(f"Attempting to analyze data from: {file_path}")
    df = pd.read_csv(file_path)

    if column not in df.columns:
      return json.dumps({
          "status": "error",
          "message": f"Column '{column}' not found in the data.",
          "available_colums": list(df.columns)
      })

    result = None

    if analysis_type == 'most frequent item':
      if pd.api.types.is_numeric_dtype(df[column]):
         # This check should be more nuanced; mode can be applied to numeric data too.
         # Let's keep it as is for now based on original code logic.
         pass
      result = df[column].mode().tolist()

    elif analysis_type == 'total volume':
      if not pd.api.types.is_numeric_dtype(df[column]):
        return json.dumps({
            "status": "error",
            "message": f"Error: 'total volume' analysis requires a numeric column, but column '{column}' is not."
        })
      result = df[column].sum()

    elif analysis_type == 'average':
      if not pd.api.types.is_numeric_dtype(df[column]):
        return json.dumps({
                    "status": "error",
                    "message": f"'average' requer coluna numérica. A coluna '{column}' não é."
                })
      result = df[column].mean()

    elif analysis_type == 'top items':
      if not top_n or top_n <= 0:
         return json.dumps({
             "status": "error",
             "message": "Você deve fornecer um valor positivo para 'top_n' para a análise de 'top items'."
         })

      # Find a non-numeric column to group by
      column = None
      for col in df.columns:
        if col != column and not pd.api.types.is_numeric_dtype(df[col]):
          group_column = col
          break

      if group_column is None:
        # If no suitable categorical column is found, raise an error
        return json.dumps({
            "status": "error",
            "message": f"Não foi possível encontrar uma coluna não numérica para agrupar e determinar 'top items' para a coluna '{column}'. 'top items' requer uma coluna de agrupamento categórica."
            })

      # Perform grouping and get top N
      grouped = df.groupby(group_column)[column].sum().nlargest(top_n)
      result = grouped.to_dict()

    else:
      return json.dumps({
          "status": "error",
          "message": f"Error: Analysis type '{analysis_type}' is not supported.",
          "supported_analyses": ['most frequent item', 'total volume', 'average', 'top items']
      })

    print(f"Successfully performed analysis: {analysis_type} on column {column}. Result: {result}")
    return json.dumps({
        "status": "success",
        "analysis_type": analysis_type,
        "column": column,
        "result": result
      })

  except FileNotFoundError:
       return json.dumps({"status": "error", "message": f"CSV file not found at '{file_path}'."})
  except Exception as e:
    return json.dumps({
            "status": "error",
            "message": f"Ocorreu um erro ao analisar os dados de '{file_path}': {str(e)}"
        })

# Add more tool functions as needed

# Setting the tools

In [3]:
available_functions = {
    "unzip_file": unzip_file,
    "read_csv": read_csv,
    "analyze_data": analyze_data,
    # Add more functions if needed
}


tools = [
    {
        "type": "function",
        "function": {
            "name": "unzip_file",
            "description": "Extract a zip file and list extracted files.",
            "parameters": {
                "type": "object",
                "properties": {
                    "zip_path": {
                        "type": "string",
                        "description": "Path to the zip file to be unzipped"
                    },
                },
                "required": ["zip_path"]
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "read_csv",
            "description": "Read and return CSV data from a folder.",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_path": {
                        "type": "string",
                        "description": "Path to the CSV file"
                    },
                },
                "required": ["file_path"]
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "analyze_data",
            "description": "Performs descriptive statistical analysis on a CSV file.",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_path": {
                        "type": "string",
                        "description": "Path to the CSV file"
                    },
                },
                "required": ["file_path"]
            },
        },
    },
    # Add more tools if needed
]

# User prompt

In [4]:
user_prompt = "Considere o arquivo 202401_NFs.zip e descompacte-o. Em seguida, analise o arquivo 202401_NFs_Itens.csv para identificar qual item teve maior volume entregue (em quantidade)." #@param {type:"string"}

# AI Agent

In [5]:
from openai import OpenAI

# Connect client
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key = API_KEY
)


# Add context about the available file and request tool usage explicitly
initial_message_content = f"""
                               Para realizar "{user_prompt}", elabore um plano passo a passo simples e execute-o usando as ferramentas disponíveis.
                               As ferramentas disponíveis são: {json.dumps(tools)}.
                               Este plano deve envolver tarefas individuais que, se executadas corretamente, resultarão na resposta correta.
                               Não adicione etapas desnecessárias.
                               O resultado da etapa final deve ser a resposta final.
                               Sempre explique o que você vai fazer antes de usar uma ferramenta.
                               Preste muita atenção aos nomes dos arquivos e parâmetros das funções.
                               Se o prompt mencionar um arquivo zip, comece descompactando-o primeiro.
                               Depois de descompactar, se o prompt mencionar um arquivo CSV específico para análise, use a ferramenta read_csv para inspecionar o arquivo e depois a ferramenta analyze_data.
                               Para a análise de 'top items', você precisará especificar a coluna a ser somada e o número de itens 'top_n'. A coluna a ser somada para 'volume entregue (em quantidade)' provavelmente será numérica e representa a quantidade. Você também precisará identificar a coluna que representa os itens (por exemplo, 'produto').
                           """

# Start AI agent loop
messages = [
  {"role": "system", "content": "Você é um assistente que usa ferramentas para processar arquivos ZIP e CSV."},
  {"role": "user", "content": initial_message_content}
]

response = client.chat.completions.create(
    model="mistralai/devstral-small:free",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

if response and response.choices:
  messages.append(response.choices[0].message)

  # Tool-calling loop
  while hasattr(response.choices[0].message, "tool_calls") and response.choices[0].message.tool_calls:
    tool_call = response.choices[0].message.tool_calls[0]
    function_name = tool_call.function.name
    try:
      arguments = json.loads(tool_call.function.arguments)
    except json.JSONDecodeError:
      error_message = "Error decoding tool call arguments."
      result = json.dumps({"error": error_message})
    else:
      if function_name in available_functions:
        result = available_functions[function_name](**arguments)
      else:
        result = json.dumps({"error": f"Function {function_name} not found."})

    messages.append({
        "role": "tool",
        "tool_call_id": tool_call.id,
        "name": function_name,
        "content": result
        })

    response = client.chat.completions.create(
        model="mistralai/devstral-small:free",
        messages=messages,
        tools=tools,
        tool_choice="auto"
        )
    if response and response.choices:
      messages.append(response.choices[0].message)

  # Final output
  if response.choices and response.choices[0].message:
    print("\nResposta final do assistente:\n")
    print(response.choices[0].message.content)
  else:
    print("\nNão foi possível obter uma resposta final do assistente.")
else:
  print("\nErro: a resposta inicial do modelo não retornou escolhas válidas.")

Attempting to unzip: 202401_NFs.zip
Successfully unzipped 202401_NFs.zip. Extracted files: ['202401_NFs_Cabecalho.csv', '202401_NFs_Itens.csv']


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '50', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1749513600000'}, 'provider_name': None}}, 'user_id': 'user_2y6pprnU0b6lrBf6Y3joeFFyUIt'}

# Perguntas
1- Considere o arquivo 202401_NFs_Itens.csv, qual é o assunto que ele possui?

R: O arquivo **202401_NFs_Itens.csv** refere-se a dados relacionados com **itens de notas fiscais** (NFs). Pela nomenclatura, podemos inferir que se trata de informações detalhadas sobre os produtos ou serviços associados a notas fiscais emitidas em **janeiro de 2024**. Isso inclui dados como:

1. **Códigos e descrições de produtos/serviços**  
2. **Quantidades**  
3. **Valores unitários e totais**  
4. **Impostos associados** (ICMS, IPI, etc.)  
5. **Chaves de acesso das notas fiscais**  
6. **Identificadores de clientes e fornecedores**  

O assunto principal é a **gestão fiscal e contábil de transações comerciais**, especificamente vinculado à estrutura de itens que compõem as notas fiscais eletrônicas (NF-e ou NFC-e). Esses dados são críticos para:  
- Controle de estoque,  
- Cálculo de tributos,  
- Auditoria fiscal,  
- Análise financeira.  

📌 **Contexto adicional**:  
O formato `.csv` indica que é uma base estruturada, provavelmente gerada por sistemas ERP (como SAP, Totvs) ou emissoras de notas fiscais (ex.: SEFAZ). O prefixo "202401" sugere periodicidade mensal (janeiro/2024), comum em relatórios fiscais e arquivos de SPED (Sistema Público de Escrituração Digital).



2- Considere o arquivo 202401_NFs_Itens.csv, qual é o fornecedor que teve maior montante recebido?

R:


3- Qual item teve maior volume entregue (em quantidade)?

R: Para identificar o item com maior volume entregue em quantidade no arquivo **202401_NFs_Itens.csv**, siga os passos abaixo:

### Passo a Passo:
1. **Leia o arquivo CSV**:  
   Carregue o conteúdo do arquivo usando uma ferramenta que permita manipulação de dados (ex: Python, Excel, R).

2. **Agrupe os itens por código e some as quantidades**:  
   - Agrupe todas as linhas pelo campo `CodItem`.  
   - Some os valores da coluna `Quant` (quantidade entregue) para cada grupo.

3. **Identifique o item com a maior soma**:  
   Encontre o registro com o valor máximo na soma das quantidades.

### Solução (usando Python com pandas):
```python
import pandas as pd

# Carregar o arquivo CSV
df = pd.read_csv("202401_NFs_Itens.csv", delimiter=';')  # Verifique o delimitador

# Agrupar por 'CodItem' e somar as quantidades
volume_por_item = df.groupby("CodItem")["Quant"].sum().reset_index()

# Encontrar o item com maior volume
item_maior_volume = volume_por_item.loc[volume_por_item["Quant"].idxmax()]

print("Item com maior volume entregue:")
print(f"Código: {item_maior_volume['CodItem']}")
print(f"Total Entregue: {item_maior_volume['Quant']}")
```

### Explicação:
- **groupby("CodItem")**: Agrupa as linhas pelo código do item.
- **["Quant"].sum()**: Soma a coluna `Quant` (quantidade) para cada grupo.
- **idxmax()**: Localiza o índice do valor máximo na coluna `Quant`.

### Resultado:
O código acima irá gerar uma saída similar a:
```
Item com maior volume entregue:
Código: [Código do Item]
Total Entregue: [Valor Máximo]
```
Substitua `[Código do Item]` e `[Valor Máximo]` pelo resultado real encontrado no arquivo.

### Observação importante:
Verifique se o arquivo utiliza **virgula**, **ponto e vírgula** ou outro delimitador. Ajuste o parâmetro `delimiter` em `pd.read_csv()` conforme necessário. Exemplo comum em dados brasileiros: `delimiter=';'`.

Para confirmar o resultado manualmente, você também pode:
1. Abrir o CSV em Excel.
2. Inserir uma tabela dinâmica:
   - Linhas: `CodItem`
   - Valores: Soma de `Quant`
3. Ordenar a coluna de soma em ordem decrescente.


4- Qual é o maior consumidor final e o número de itens comprados?

R: Para determinar o maior consumidor final (pessoa física que compra para uso próprio) e o número total de itens comprados, siga as etapas no código Python abaixo. O arquivo CSV `202401_NFs_Itens.csv` deve conter uma coluna que identifique o tipo de cliente (ex.: `Tipo_Cliente` com valor `'Consumidor Final'`), um identificador único para o cliente (ex.: `CPF`) e a quantidade de itens por registro.

```python
import pandas as pd

# Carregar o arquivo CSV (ajuste sep/encoding conforme necessário)
df = pd.read_csv('202401_NFs_Itens.csv', sep=';', encoding='latin1')

# Verificar colunas (para depuração)
# print(df.columns.tolist())

# Pressupostos sobre as colunas:
#   - Coluna de tipo de cliente: 'Tipo_Cliente'
#   - Coluna de identificação do cliente: 'CPF'
#   - Coluna de quantidade: 'Quantidade'

# Filtrar apenas consumidores finais
df_consumidor_final = df[df['Tipo_Cliente'] == 'Consumidor Final']

# Agrupar por CPF e somar as quantidades
compras_por_cliente = df_consumidor_final.groupby('CPF')['Quantidade'].sum()

# Identificar o maior consumidor
maior_consumidor_id = compras_por_cliente.idxmax()
total_itens = compras_por_cliente.max()

print(f"Maior consumidor final: CPF {maior_consumidor_id}")
print(f"Total de itens comprados: {total_itens}")
```

### Explicação:
1. **Filtragem de Consumidores Finais**: Apenas registros marcados como `'Consumidor Final'` são considerados.
2. **Agregação**: Itens são somados por cliente (usando `CPF` como identificador).
3. **Identificação do Maior Consumidor**: O cliente com a maior soma de itens é extraído.

### Observações:
- **Colunas do arquivo**: Se as colunas no CSV tiverem nomes diferentes (ex.: `CNPJ_CPF`, `Qtde`), ajuste os nomes no código.
- **Formato do arquivo**: Se o CSV usar separadores diferentes (ex.: vírgula), ajuste `sep` (ex.: `sep=','`).
- **Resultado**: A saída será o CPF do maior consumidor e o total de itens comprados por ele. Se houver empate, retornará o primeiro cliente com a quantidade máxima.

### Execução:
Execute o código com o arquivo no mesmo diretório. Se precisar de detalhes da estrutura do arquivo, descomente `print(df.columns.tolist())`.

Resultado esperado (exemplo):
```
Maior consumidor final: CPF 123.456.789-00
Total de itens comprados: 150
```